In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.nhl_db
collection = db.articles

In [4]:
# URL of page to be scraped
url = 'https://www.nhl.com/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [9]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en_US">
 <head>
  <title>
   Official Site of the National Hockey League | NHL.com
  </title>
  <!-- meta meta tag -->
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="no-cache" http-equiv="Cache-Control"/>
  <meta content="no-cache" http-equiv="Pragma"/>
  <meta content="-1" http-equiv="Expires"/>
  <meta content="en" http-equiv="content-language"/>
  <meta content="nhl, nhl.com, www.nhl.com, playoffs, scores, video, photos, standings, news, features, players, shop, auctions, tickets, mobile, game center live, stanley cup, winter classic, draft, free agency" name="keywords"/>
  <meta content="Official Site of the National Hockey League" property="og:title"/>
  <meta content="Official Site of the National Hockey League NHL.com" itemprop="name"/>
  <meta content="NHL.com" property="og:site_name"/>
  <meta content="website" property="og:type"/>
  <meta content="http

In [11]:
results = soup.find_all('li', class_='mixed-feed__item--article')
results

[<li class="mixed-feed__item mixed-feed__item--article" data-content-id="299744328">
 <div class="mixed-feed__item-header">
 <div class="mixed-feed__logos">
 <div class="mixed-feed__team-logo mixed-feed__team-logo-nhl logo-round-team logo-bg-dark--league-nhl primary-bg--league-nhl"></div>
 </div>
 <div class="mixed-feed__item-header-text">
 <a href="/news/2018-preseason-schedule/c-299744328">
 <h4 class="mixed-feed__header headline-link">NHL China Games highlight preseason schedule</h4>
 <h5 class="mixed-feed__subheader">League also to play in Germany, Switzerland, plus Kraft Hockeyville in Ontario and New York</h5>
 </a>
 </div>
 </div>
 <div class="mixed-feed__item-content">
 <div class="mixed-feed__meta">
 <div class="mixed-feed__tags">
 <a class="mixed-feed__tag-link" disabled="true">Press Release</a>
 </div>
 <a href="/news/2018-preseason-schedule/c-299744328">
 <h4 class="mixed-feed__header-meta">
                                                     NHL China Games highlight pres

In [5]:
# Retrieve the parent divs for all articles
results = soup.find_all('li', class_='mixed-feed__item--article')

# Loop through results to retrieve article title, header, and timestamp of article
for result in results:
    title = result.find('h4', class_='mixed-feed__header').text

    lede = result.find('h5', class_='mixed-feed__subheader').text

    # The time and date of article publication
    date = result.find('time')['datetime']
    # Slice the datetime string for the date
    article_date = date[:10]
    # Slice the datetime string for the time
    time = date[11:16]
    # Determine whether article was published in AM or PM
    if (int(time[:2]) >= 13):
        meridiem = 'pm'
    else:
        meridiem = 'am'

    # Concatenate time string
    time = time + meridiem
    print('-----------------')
    print(title)
    print(lede)
    print(article_date)
    print(time)

    # Dictionary to be inserted into MongoDB
    post = {
        'title': title,
        'lede': lede,
        'date': article_date,
        'time published': time
    }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)

-----------------
Lady Byng Trophy finalists unveiled
Barkov, O'Reilly, William Karlsson up for award given for combination of skill, gentlemanly play
2018-04-20
19:00pm
-----------------
Peters resigns as Hurricanes coach
Went 137-138-53 in four seasons with Carolina
2018-04-20
12:00am
-----------------
Bergeron skates, could return for Bruins in Game 5 against Maple Leafs
Boston one win from advancing to second round
2018-04-20
14:33pm
-----------------
Friday Four: Golden Knights continue fairy-tale season
NHL Network analyst Weekes also discusses Sharks, Capitals-Blue Jackets series, Subban
2018-04-20
14:01pm
-----------------
Fantasy: Daily primer for 2018 Stanley Cup Playoffs
Kadri back from suspension in Game 5; Bergeron could return; Wennberg skating
2018-04-20
19:05pm
-----------------
Sharks expect to stick with same lineup in second round
Coach says he'd be 'idiot' to make changes from group that swept Ducks
2018-04-20
17:20pm
-----------------
Doughty open to signing extens

In [6]:
# Display the MongoDB records created above
articles = db.articles.find()
for article in articles:
    print(article)

{'_id': ObjectId('5ada7a1bee61f93d3f7e3d07'), 'title': 'Lady Byng Trophy finalists unveiled', 'lede': "Barkov, O'Reilly, William Karlsson up for award given for combination of skill, gentlemanly play", 'date': '2018-04-20', 'time published': '19:00pm'}
{'_id': ObjectId('5ada7a1bee61f93d3f7e3d08'), 'title': 'Peters resigns as Hurricanes coach', 'lede': 'Went 137-138-53 in four seasons with Carolina', 'date': '2018-04-20', 'time published': '12:00am'}
{'_id': ObjectId('5ada7a1bee61f93d3f7e3d09'), 'title': 'Bergeron skates, could return for Bruins in Game 5 against Maple Leafs', 'lede': 'Boston one win from advancing to second round', 'date': '2018-04-20', 'time published': '14:33pm'}
{'_id': ObjectId('5ada7a1bee61f93d3f7e3d0a'), 'title': 'Friday Four: Golden Knights continue fairy-tale season', 'lede': 'NHL Network analyst Weekes also discusses Sharks, Capitals-Blue Jackets series, Subban', 'date': '2018-04-20', 'time published': '14:01pm'}
{'_id': ObjectId('5ada7a1bee61f93d3f7e3d0b'), '